# Previsão de Senioridade (Júnior / Pleno / Sênior)
Notebook mais robusto para carregar os dados, pré-processar as habilidades e treinar uma Regressão Logística multinomial.

**Observações:** este notebook tenta detectar automaticamente como as habilidades estão formatadas e trata valores ausentes.

In [ ]:
# 1) Carregar base de dados (tratando possíveis problemas de encoding)
import pandas as pd
from pathlib import Path

file_path = Path('/mnt/data/job_skills.csv')
print('Arquivo existe?', file_path.exists())

# tentar leituras com encodings comuns se necessário
encodings_to_try = ['utf-8', 'latin-1', 'cp1252']
for enc in encodings_to_try:
    try:
        df = pd.read_csv(file_path, encoding=enc)
        print(f'Arquivo carregado com encoding: {enc} — shape:', df.shape)
        break
    except Exception as e:
        last_err = e
        df = None
else:
    raise last_err

# mostrar colunas e primeiras linhas
print('\nColunas encontradas:', df.columns.tolist())
df.head()

In [ ]:
# 2) Inspeção rápida de colunas relevantes e limpeza básica
# Procurar colunas típicas: 'title', 'skills', 'seniority', 'salario', 'salary'
possible_skills_cols = [c for c in df.columns if 'skill' in c.lower() or 'habil' in c.lower()]
possible_seniority_cols = [c for c in df.columns if 'senior' in c.lower() or 'level' in c.lower()]

print('Coluna(s) candidatas a skills:', possible_skills_cols)
print('Coluna(s) candidatas a seniority:', possible_seniority_cols)

# Escolher colunas (prioridade: detected lists, senority)
skills_col = possible_skills_cols[0] if possible_skills_cols else None
seniority_col = possible_seniority_cols[0] if possible_seniority_cols else None
print('Usando skills_col =', skills_col, 'e seniority_col =', seniority_col)

# Se não existirem, exibir primeiras linhas para o usuário revisar
if skills_col is None or seniority_col is None:
    print('\nNenhuma coluna clara detectada para skills ou seniority. Exibindo primeiras 10 linhas para inspeção:')
    display(df.head(10))

In [ ]:
# 3) Normalizar e extrair lista de skills (tratando formatos variados)
import re
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import chain

def extract_skills(cell):
    if pd.isna(cell):
        return []
    # Se já é lista-like (string com colchetes), remover e splitar
    text = str(cell)
    # remover parênteses e conteúdo dentro se for muito verboso
    text = re.sub(r'\(.*?\)', '', text)
    # dividir por vírgula, ponto-e-vírgula, barra, pipe ou '|'
    parts = re.split(r',|;|\/|\||\\\n|\\n', text)
    # further split by 'and' or '/' if no delimiter found
    parts = [p.strip() for p in parts if p and p.strip()!='']
    # separar tokens compostos por espaços quando necessário? NÃO, manter frases como 'machine learning'
    # normalizar lower and strip
    parts = [re.sub('\s+', ' ', p).strip().lower() for p in parts]
    # remover itens muito curtos
    parts = [p for p in parts if len(p) > 1]
    return parts

# aplicar coluna skills
if skills_col is None:
    raise ValueError('Não foi possível detectar automaticamente a coluna de skills. Por favor renomeie a coluna de habilidades para incluir "skill" ou "habil".')

df['skills_list'] = df[skills_col].apply(extract_skills)
# mostrar exemplos
df[['skills_list']].head(10)

In [ ]:
# 4) Binarizar skills com MultiLabelBinarizer e filtrar skills raras
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=False)
skills_bin = mlb.fit_transform(df['skills_list'])
skills_df = pd.DataFrame(skills_bin, columns=mlb.classes_, index=df.index)

# remover skills muito raras (< 1% das vagas) para reduzir dimensionalidade
min_freq = max(1, int(0.01 * len(skills_df)))
freq = skills_df.sum(axis=0)
keep_cols = freq[freq >= min_freq].index.tolist()
print(f'Total skills originais: {len(skills_df.columns)} — mantendo {len(keep_cols)} com frequência >= {min_freq}')
skills_df = skills_df[keep_cols]
skills_df.head()

In [ ]:
# 5) Criar variável-alvo (mapear seniority para 0=Junior,1=Pleno,2=Senior quando possível)
if seniority_col is None:
    raise ValueError('Não foi possível detectar automaticamente a coluna de senioridade. Por favor renomeie a coluna de senioridade para incluir "senior" ou "level".')

# Normalizar valores de seniority comuns
def map_seniority(val):
    if pd.isna(val):
        return None
    s = str(val).strip().lower()
    if any(k in s for k in ['jr', 'junior']):
        return 0
    if any(k in s for k in ['pleno', 'mid', 'mid-level', 'mid level']):
        return 1
    if any(k in s for k in ['senior', 'sr', 'sênior', 'sénior']):
        return 2
    # casos numéricos (ex: '3' years?) tentamos não inferir
    return None

df['target'] = df[seniority_col].apply(map_seniority)
print('Contagem dos targets (antes de remover NAs):')
print(df['target'].value_counts(dropna=False))

# remover linhas sem target ou sem skills binárias
mask = df['target'].notna() & (skills_df.sum(axis=1) > 0)
df_model = df[mask].copy()
skills_df_model = skills_df.loc[mask]
df_model['target'] = df_model['target'].astype(int)
print('\nApós filtragem — linhas disponíveis para modelagem:', df_model.shape[0])

In [ ]:
# 6) Modelagem: Regressão Logística Multinomial
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

X = skills_df_model
y = df_model['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = LogisticRegression(max_iter=2000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print('Relatório de classificação (test set):\n')
print(classification_report(y_test, y_pred, target_names=['Junior','Pleno','Senior']))

# matriz de confusão
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=['Junior','Pleno','Senior'])
fig, ax = plt.subplots(figsize=(6,6))
disp.plot(ax=ax)
plt.title('Matriz de Confusão - Regressão Logística')
plt.show()

In [ ]:
# 7) Interpretabilidade: top features por classe (coeficientes do modelo)
import numpy as np

coefs = model.coef_  # shape (n_classes, n_features)
feature_names = X.columns.tolist()

for class_idx, class_name in enumerate(['Junior','Pleno','Senior']):
    top_pos_idx = np.argsort(coefs[class_idx])[-10:][::-1]
    top_neg_idx = np.argsort(coefs[class_idx])[:10]
    print(f'\nTop 10 habilidades associadas a classe {class_name} (pos):')
    for i in top_pos_idx:
        print(f'  {feature_names[i]} ({coefs[class_idx,i]:.3f})')
    print(f'\nTop 10 habilidades associadas negativamente a classe {class_name} (neg):')
    for i in top_neg_idx:
        print(f'  {feature_names[i]} ({coefs[class_idx,i]:.3f})')

## Discussão e próximos passos
- Validar qualidade dos rótulos de senioridade (muitas vagas não explicitam claramente)
- Considerar balanceamento (SMOTE) se classes desbalanceadas
- Usar modelos mais complexos (RandomForest, XGBoost) e comparar
- Construir pipeline completo e salvar o modelo

Se preferir, eu posso:
- Rodar este notebook agora e mostrar os resultados aqui;
- Ajustar thresholds, incluir validação cruzada e métricas adicionais;
- Gerar relatório PDF com interpretações e recomendações de carreira.